In [ ]:
import pandas as pd
import numpy as np
import warnings
import matplotlib as mpl
from datetime import datetime as dt
from darts import TimeSeries
from darts.models import(
        ExponentialSmoothing,
        ARIMA,
#         Prophet,
        Theta
        )
from darts.metrics import mae, mase, rmse
from darts.utils.statistics import check_seasonality, plot_acf

In [2]:
# Shoot a warning that is generated by the Theta model for too high values of theta whilst finetuning
warnings.filterwarnings('ignore', message='Optimization failed to converge. Check mle_retvals.')

# Pre-processing of the data

In [3]:
# sort the dates out because they weird. drop x2 because not useful. Dates that were in index make a column. 
# and we didn't like the original names

In [4]:
data = pd.read_csv('M3_in_random.csv', header=0)
data_test = pd.read_csv('M3_out_random.csv', header=0)

In [5]:
data.drop('X2', axis=1, inplace=True)

data = data.transpose()

data.reset_index(inplace=True, drop=True)
data.columns = [f'x{i+1}' for i in range(len(data.columns))]
data = data.iloc[1:,:]

data['Date'] = pd.date_range("2000", freq='Y', periods=data.shape[0])

In [6]:
data_test.drop('X2', axis=1, inplace=True)

data_test = data_test.transpose()

data_test.reset_index(inplace=True, drop=True)
data_test.columns = [f'x{i+1}' for i in range(len(data_test.columns))]
data_test = data_test.iloc[1:7,:]

In [7]:
data

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x211,x212,x213,x214,x215,x216,x217,x218,x219,Date
1,1169.0,6182.0,5275.58,3525.2,360.70,3000.0,4631.45,3650.0,3011.0,3574.5,...,8050.0,5314.0,1874.0,3985.0,4385.7,4478.0,1005.0,1751.0,2283.7,2000-12-31
2,1563.0,6191.5,5832.80,3711.2,436.22,3820.0,2885.05,4225.0,2979.0,3690.0,...,8020.0,5592.0,2837.0,4578.0,4486.6,4663.0,900.0,1681.0,2363.4,2001-12-31
3,1999.0,6198.5,5815.04,4035.0,543.50,4720.0,2136.50,2940.0,3145.0,3921.0,...,8010.0,5063.0,4034.0,5224.0,4375.4,4249.0,1395.0,1704.0,2345.6,2002-12-31
4,2530.0,6292.5,5981.24,4349.8,622.74,3500.0,5478.80,3305.0,3209.0,4130.0,...,8030.0,5410.0,4967.0,5294.0,4519.7,4252.0,3135.0,1698.0,2392.9,2003-12-31
5,3214.0,6535.0,5905.00,4655.4,723.82,7120.0,2805.60,6605.0,3028.0,4070.0,...,8490.0,5660.0,6527.0,6118.0,4781.9,4694.0,3285.0,1680.0,2503.7,2004-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2121-12-31
123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2122-12-31
124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2123-12-31
125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2124-12-31


In [8]:
data_test

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x210,x211,x212,x213,x214,x215,x216,x217,x218,x219
1,7668.0,7845.0,5939.64,6359.4,4138.50,7100.0,1443.55,6030.0,4952.0,5614.0,...,5839.5,8270.0,4069.0,7886.0,7985.0,10521.0,3387.0,5850.0,5421.0,6851.0
2,7800.0,7742.0,6334.18,6728.4,4856.04,6000.0,1550.10,5810.0,5029.0,6031.0,...,6011.0,8190.0,4126.0,8152.0,8117.0,10789.5,3481.0,5880.0,5640.0,7072.6
3,7940.0,7566.0,1558.00,6956.8,5763.38,4980.0,1359.85,3700.0,5120.0,6232.5,...,6206.5,8110.0,4032.0,8684.0,7909.0,10941.9,3466.0,8520.0,5322.0,7203.9
4,8074.0,7384.0,5533.68,7141.6,6722.46,6540.0,1134.10,3395.0,5324.0,6274.0,...,6720.5,8110.0,4040.0,8890.0,8173.0,10825.6,3411.0,9450.0,5289.0,7144.8
5,8219.0,7456.0,5446.26,7113.2,7895.96,6880.0,1559.05,1895.0,5304.0,6145.0,...,7586.0,7840.0,3927.0,8984.0,8758.0,10851.9,3267.0,7035.0,4661.0,7182.6
6,8371.0,7672.0,6532.78,7253.2,8313.90,6700.0,2490.00,2770.0,5251.0,6156.0,...,7943.5,7650.0,4123.0,10846.0,8160.0,11017.8,3039.0,9960.0,4579.0,7353.4


# Is there seasonality ?

In [9]:
seasonality = dict()
for column in data.columns[:-1]:
    temp = data.loc[:,[column,'Date']]
    temp.dropna(inplace=True)
    data_ts = TimeSeries.from_dataframe(temp, time_col='Date')
    
    seasonal_periods = []
    for m in range (2, 25):
        is_seasonal, period = check_seasonality(data_ts, m=m, alpha=.05)
        if is_seasonal:
            seasonal_periods.append(period)
    try :
        seasonality[column] = seasonal_periods[0]
    except IndexError:
        seasonality[column] = 1
    seasonal_periods = []
    
seasonality

[2021-12-11 19:02:29,656] INFO | darts.utils.statistics | The ACF has no local maximum for m < max_lag = 24.
[2021-12-11 19:02:29,658] INFO | darts.utils.statistics | The ACF has no local maximum for m < max_lag = 24.
[2021-12-11 19:02:29,659] INFO | darts.utils.statistics | The ACF has no local maximum for m < max_lag = 24.
[2021-12-11 19:02:29,661] INFO | darts.utils.statistics | The ACF has no local maximum for m < max_lag = 24.
[2021-12-11 19:02:29,662] INFO | darts.utils.statistics | The ACF has no local maximum for m < max_lag = 24.
[2021-12-11 19:02:29,664] INFO | darts.utils.statistics | The ACF has no local maximum for m < max_lag = 24.
[2021-12-11 19:02:29,666] INFO | darts.utils.statistics | The ACF has no local maximum for m < max_lag = 24.
[2021-12-11 19:02:29,667] INFO | darts.utils.statistics | The ACF has no local maximum for m < max_lag = 24.
[2021-12-11 19:02:29,669] INFO | darts.utils.statistics | The ACF has no local maximum for m < max_lag = 24.
[2021-12-11 19:02:2

{'x1': 1,
 'x2': 1,
 'x3': 3,
 'x4': 1,
 'x5': 1,
 'x6': 3,
 'x7': 5,
 'x8': 9,
 'x9': 1,
 'x10': 1,
 'x11': 1,
 'x12': 1,
 'x13': 1,
 'x14': 1,
 'x15': 7,
 'x16': 1,
 'x17': 1,
 'x18': 1,
 'x19': 1,
 'x20': 4,
 'x21': 1,
 'x22': 1,
 'x23': 1,
 'x24': 1,
 'x25': 1,
 'x26': 1,
 'x27': 1,
 'x28': 1,
 'x29': 1,
 'x30': 1,
 'x31': 10,
 'x32': 6,
 'x33': 22,
 'x34': 1,
 'x35': 1,
 'x36': 1,
 'x37': 3,
 'x38': 1,
 'x39': 1,
 'x40': 1,
 'x41': 6,
 'x42': 1,
 'x43': 1,
 'x44': 1,
 'x45': 1,
 'x46': 1,
 'x47': 12,
 'x48': 1,
 'x49': 1,
 'x50': 1,
 'x51': 1,
 'x52': 17,
 'x53': 1,
 'x54': 1,
 'x55': 1,
 'x56': 1,
 'x57': 1,
 'x58': 1,
 'x59': 3,
 'x60': 1,
 'x61': 23,
 'x62': 1,
 'x63': 1,
 'x64': 1,
 'x65': 1,
 'x66': 1,
 'x67': 17,
 'x68': 1,
 'x69': 1,
 'x70': 1,
 'x71': 1,
 'x72': 1,
 'x73': 6,
 'x74': 8,
 'x75': 1,
 'x76': 1,
 'x77': 1,
 'x78': 1,
 'x79': 1,
 'x80': 17,
 'x81': 7,
 'x82': 3,
 'x83': 1,
 'x84': 11,
 'x85': 1,
 'x86': 5,
 'x87': 1,
 'x88': 1,
 'x89': 1,
 'x90': 1,
 'x91': 1,


# Create the modelisation object

In [10]:
class Modelise():
    def __init__(self, model, train, validation,  **kwargs):
        self.model = model
        self.train = train
        self.validation = validation
        self.__dict__.update(kwargs)
        self.model.fit(self.train)

        return None

    def predict(self):
        forecast = self.model.predict(len(self.validation))

        return forecast

    def performance(self):
        error_mae = mae(self.validation, forecast)
        error_mase = mase(self.validation, forecast, self.train)
        error_rmse = rmse(self.validation, forecast)

        return error_mae, error_mase, error_rmse

# Iterate over each column of the M3 dataset to select the best model with a validation subset (1 column = 1 product)

In [11]:
model_forecast_matrix = pd.DataFrame(index=range(15))

model_comparison_matrix = pd.DataFrame(index=['error_mae', 'error_mase', 'error_rmse'])

best_model = {}

In [ ]:
for column in data.columns[:-1]:
    temp = data.loc[:,[column,'Date']]
    temp.dropna(inplace=True)
    data_ts = TimeSeries.from_dataframe(temp, time_col='Date')
    
    slicer = round(0.8*temp.shape[0])
    slicer_date = data.loc[slicer,'Date']
    train, validation = data_ts.split_before(slicer_date)
    
    # Exponential smoothing
    try:
        if seasonality[f'{column}'] == 1:
            model_exp = Modelise(ExponentialSmoothing(seasonal_periods=temp.shape[0]), train, validation)
        else:
            model_exp = Modelise(ExponentialSmoothing(seasonal_periods=seasonality[f'{column}']), train, validation)
        forecast = model_exp.predict()
        error_mae, error_mase, error_rmse = model_exp.performance()

        model_comparison_matrix['exp_smooth'] = (error_mae, error_mase, error_rmse)
        model_forecast_matrix['exp_smooth'] = forecast.values()
    except ValueError:
        model_comparison_matrix['exp_smooth'] = np.nan
        model_forecast_matrix['exp_smooth'] = np.nan
        
    # ARIMA
    try:
        model_arima = Modelise(ARIMA(), train, validation)
        forecast = model_arima.predict()
        error_mae, error_mase, error_rmse = model_arima.performance()

        model_comparison_matrix['arima'] = (error_mae, error_mase, error_rmse)
        model_forecast_matrix['arima'] = forecast.values()
    except ValueError:
        model_comparison_matrix['arima'] = np.nan
        model_forecast_matrix['arima'] = np.nan
        
    # Theta Method
    optimal_theta = 0
    best_error_rmse = float('inf')
    finetune_theta = dict()
    try:
        for theta_ in np.linspace(-4, 4, 200):
            temp_model = Modelise(Theta(theta_, seasonality_period=seasonality[f'{column}']),
                    train,
                    validation)
            forecast = temp_model.predict()
            error_mae, error_mase, error_rmse = temp_model.performance()
            finetune_theta[theta_] = (error_mae, error_mase, error_rmse) 
            if error_rmse < best_error_rmse:
                best_error_rmse = error_rmse
                optimal_theta = theta_

        model_theta = Modelise(Theta(optimal_theta, seasonality_period=seasonality[f'{column}']), train, validation)
        forecast = model_theta.predict()
        error_mae, error_mase, error_rmse = model_theta.performance()
        model_comparison_matrix['theta'] = (error_mae, error_mase, error_rmse)
#     model_forecast_matrix[f'theta_{column}'] = forecast.values()
    except ValueError:
        model_comparison_matrix['theta'] = np.nan
        
    best_model[f'{column}'] = model_comparison_matrix.loc['error_mae'].idxmin()

[2021-12-11 19:02:43,043] ERROR | main_logger | ValueError: Train series only contains 14 elements but Exponential smoothing model requires at least 38 entries
[2021-12-11 19:02:43,043] ERROR | main_logger | ValueError: Train series only contains 14 elements but Exponential smoothing model requires at least 38 entries
[2021-12-11 19:02:43,043] ERROR | main_logger | ValueError: Train series only contains 14 elements but Exponential smoothing model requires at least 38 entries
[2021-12-11 19:02:43,043] ERROR | main_logger | ValueError: Train series only contains 14 elements but Exponential smoothing model requires at least 38 entries
[2021-12-11 19:02:43,049] ERROR | main_logger | ValueError: Train series only contains 14 elements but ARIMA(12, 1, 0) model requires at least 30 entries
[2021-12-11 19:02:43,049] ERROR | main_logger | ValueError: Train series only contains 14 elements but ARIMA(12, 1, 0) model requires at least 30 entries
[2021-12-11 19:02:43,049] ERROR | main_logger | Valu

In [ ]:
best_model

# Forecast for the next 5 time points

In [ ]:
data_test

In [ ]:
result_forecast_matrix = pd.DataFrame(index=range(1,7))

result_comparison_matrix = pd.DataFrame(index=['error_mae', 'error_mase', 'error_rmse'])

In [ ]:
for column in data.columns[:-1]:
    temp = data.loc[:,[column,'Date']]
    temp.dropna(inplace=True)
    train = TimeSeries.from_dataframe(temp, time_col='Date')
    
    data_test['Date'] = pd.date_range(str(2000+temp.shape[0]), freq='Y', periods=6)
    temp2 = data_test.loc[:,[column,'Date']]
    test = TimeSeries.from_dataframe(temp2, time_col='Date')
    
    result_forecast_matrix[f'real_data_{column}'] = temp2[column]
    
    if best_model[f'{column}'] == 'exp_smooth':
        if seasonality[f'{column}'] == 1:
            model_exp = Modelise(ExponentialSmoothing(seasonal_periods=temp.shape[0]), train, validation)
        else:
            model_exp = Modelise(ExponentialSmoothing(seasonal_periods=seasonality[f'{column}']), train, validation)
        forecast = model_exp.predict()
        error_mae, error_mase, error_rmse = model_exp.performance()

        result_comparison_matrix[f'exp_smooth_{column}'] = (error_mae, error_mase, error_rmse)
        result_forecast_matrix[f'exp_smooth_{column}'] = forecast.values()
    
    if best_model[f'{column}'] == 'arima':
        model_arima = Modelise(ARIMA(), train, test)
        forecast = model_arima.predict()
        error_mae, error_mase, error_rmse = model_arima.performance()

        result_comparison_matrix[f'arima_{column}'] = (error_mae, error_mase, error_rmse)
        result_forecast_matrix[f'arima_{column}'] = forecast.values()
        
    if best_model[f'{column}'] == 'theta':
        optimal_theta = 0
        best_error_rmse = float('inf')
        finetune_theta = dict()
        for theta_ in np.linspace(-4, 4, 200):
            temp_model = Modelise(Theta(theta_, seasonality_period=seasonality[f'{column}']),
                    train,
                    test)
            forecast = temp_model.predict()
            error_mae, error_mase, error_rmse = temp_model.performance()
            finetune_theta[theta_] = (error_mae, error_mase, error_rmse) 
            if error_rmse < best_error_rmse:
                best_error_rmse = error_rmse
                optimal_theta = theta_

        model_theta = Modelise(Theta(optimal_theta, seasonality_period=seasonality[f'{column}']), train, test)
        forecast = model_theta.predict()
        error_mae, error_mase, error_rmse = model_theta.performance()
        result_comparison_matrix[f'theta_{column}'] = (error_mae, error_mase, error_rmse)
        result_forecast_matrix[f'theta_{column}'] = forecast.values()
    else:
        pass

In [ ]:
result_comparison_matrix

In [ ]:
result_forecast_matrix

In [ ]:
result_forecast_matrix[['real_data_x1', 'theta_x1']].plot()

In [ ]:
result_forecast_matrix[['real_data_x2', 'theta_x2']].plot()

In [ ]:
for n in range(1,21):
    try:
        fig = result_forecast_matrix[[f'real_data_x{n}', f'theta_x{n}']].plot()
        fig.savefig(f'x{n}.png')
    except:
        pass